<a href="https://colab.research.google.com/github/weasel-codes/covid-patient-recovery/blob/main/CV_RESNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
import pandas as pd
import os
import shutil
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [2]:
def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 3
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    
    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

In [3]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)

    # Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = (f, f), strides = (1,1), padding = 'same', name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)


    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)


    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = (1, 1), strides = (s,s), padding = 'valid', name = conv_name_base + '1',
                        kernel_initializer = glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name = bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    
    return X

# Building Model

In [4]:
def ResNet50(input_shape=(64, 64, 3), classes=6):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), name='conv1', kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name='bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block='a', s=1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], stage = 3, block='a', s = 2)
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='b')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='c')
    X = identity_block(X, 3, [128, 128, 512], stage=3, block='d')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], stage = 4, block='a', s = 2)
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='b')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='c')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='d')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='e')
    X = identity_block(X, 3, [256, 256, 1024], stage=4, block='f')

    # Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], stage = 5, block='a', s = 2)
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='b')
    X = identity_block(X, 3, [512, 512, 2048], stage=5, block='c')

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D((2,2), name="avg_pool")(X)

    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [5]:
model = ResNet50(input_shape = (64, 64, 3), classes = 2)
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 32, 64)   9472        zero_padding2d[0][0]             
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 32, 32, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

In [6]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
import tensorflow as tf
from random import shuffle 
from tqdm import tqdm 
import cv2 
from numpy import load

In [9]:
width, height = 64,64
covid_path = "/content/drive/MyDrive/ML/cv/covid/"
normal_path = "/content/drive/MyDrive/ML/cv/normal/"
img_size = (width, height, 3)

In [11]:
image_names=os.listdir(covid_path)
image_names[:5]

['COVID-100.png',
 'COVID-1.png',
 'COVID-10.png',
 'COVID-102.png',
 'COVID-101.png']

In [115]:
data_covid = []
data_normal = []

for img in tqdm(os.listdir(normal_path)):
  path = os.path.join(normal_path, img)
  print(path)

  # loading the image from the path and then converting them into 
  # greyscale for easier covnet prob 
  img = cv2.imread(path, cv2.IMREAD_COLOR)
  img = cv2.resize(img, (64, 64))
  img = img/255
  data_normal.append([img, 1])

for img in tqdm(os.listdir(covid_path)): 
  path = os.path.join(covid_path, img)
  print(path)

  # loading the image from the path and then converting them into 
  # greyscale for easier covnet prob 
  img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
  img = cv2.resize(img, (width, height))
  img = img/255
  # final step-forming the training data list with numpy array of the images 
  data_covid.append([img, 0])


  6%|▌         | 29/500 [00:00<00:01, 284.95it/s]

/content/drive/MyDrive/ML/cv/normal/Normal-1.png
/content/drive/MyDrive/ML/cv/normal/Normal-100.png
/content/drive/MyDrive/ML/cv/normal/Normal-10.png
/content/drive/MyDrive/ML/cv/normal/Normal-101.png
/content/drive/MyDrive/ML/cv/normal/Normal-102.png
/content/drive/MyDrive/ML/cv/normal/Normal-103.png
/content/drive/MyDrive/ML/cv/normal/Normal-104.png
/content/drive/MyDrive/ML/cv/normal/Normal-105.png
/content/drive/MyDrive/ML/cv/normal/Normal-106.png
/content/drive/MyDrive/ML/cv/normal/Normal-108.png
/content/drive/MyDrive/ML/cv/normal/Normal-107.png
/content/drive/MyDrive/ML/cv/normal/Normal-109.png
/content/drive/MyDrive/ML/cv/normal/Normal-11.png
/content/drive/MyDrive/ML/cv/normal/Normal-110.png
/content/drive/MyDrive/ML/cv/normal/Normal-112.png
/content/drive/MyDrive/ML/cv/normal/Normal-111.png
/content/drive/MyDrive/ML/cv/normal/Normal-113.png
/content/drive/MyDrive/ML/cv/normal/Normal-114.png
/content/drive/MyDrive/ML/cv/normal/Normal-115.png
/content/drive/MyDrive/ML/cv/normal

 18%|█▊        | 88/500 [00:00<00:01, 285.02it/s]

/content/drive/MyDrive/ML/cv/normal/Normal-151.png
/content/drive/MyDrive/ML/cv/normal/Normal-152.png
/content/drive/MyDrive/ML/cv/normal/Normal-154.png
/content/drive/MyDrive/ML/cv/normal/Normal-153.png
/content/drive/MyDrive/ML/cv/normal/Normal-156.png
/content/drive/MyDrive/ML/cv/normal/Normal-157.png
/content/drive/MyDrive/ML/cv/normal/Normal-155.png
/content/drive/MyDrive/ML/cv/normal/Normal-162.png
/content/drive/MyDrive/ML/cv/normal/Normal-160.png
/content/drive/MyDrive/ML/cv/normal/Normal-161.png
/content/drive/MyDrive/ML/cv/normal/Normal-158.png
/content/drive/MyDrive/ML/cv/normal/Normal-16.png
/content/drive/MyDrive/ML/cv/normal/Normal-159.png
/content/drive/MyDrive/ML/cv/normal/Normal-163.png
/content/drive/MyDrive/ML/cv/normal/Normal-164.png
/content/drive/MyDrive/ML/cv/normal/Normal-165.png
/content/drive/MyDrive/ML/cv/normal/Normal-168.png
/content/drive/MyDrive/ML/cv/normal/Normal-167.png
/content/drive/MyDrive/ML/cv/normal/Normal-166.png
/content/drive/MyDrive/ML/cv/nor

 35%|███▍      | 173/500 [00:00<00:01, 280.48it/s]

...

/content/drive/MyDrive/ML/cv/covid/COVID-402.png
/content/drive/MyDrive/ML/cv/covid/COVID-387.png
/content/drive/MyDrive/ML/cv/covid/COVID-369.png
/content/drive/MyDrive/ML/cv/covid/COVID-395.png
/content/drive/MyDrive/ML/cv/covid/COVID-400.png
/content/drive/MyDrive/ML/cv/covid/COVID-405.png
/content/drive/MyDrive/ML/cv/covid/COVID-37.png
/content/drive/MyDrive/ML/cv/covid/COVID-370.png
/content/drive/MyDrive/ML/cv/covid/COVID-411.png
/content/drive/MyDrive/ML/cv/covid/COVID-380.png
/content/drive/MyDrive/ML/cv/covid/COVID-388.png
/content/drive/MyDrive/ML/cv/covid/COVID-394.png
/content/drive/MyDrive/ML/cv/covid/COVID-389.png
/content/drive/MyDrive/ML/cv/covid/COVID-412.png
/content/drive/MyDrive/ML/cv/covid/COVID-371.png
/content/drive/MyDrive/ML/cv/covid/COVID-406.png
/content/drive/MyDrive/ML/cv/covid/COVID-409.png
/content/drive/MyDrive/ML/cv/covid/COVID-39.png
/content/drive/MyDrive/ML/cv/covid/COVID-393.png
/content/drive/MyDrive/ML/cv/covid/COVID-401.png
/content/drive/MyDrive

 85%|████████▌ | 426/500 [00:01<00:00, 295.93it/s]

/content/drive/MyDrive/ML/cv/covid/COVID-43.png
/content/drive/MyDrive/ML/cv/covid/COVID-42.png
/content/drive/MyDrive/ML/cv/covid/COVID-421.png
/content/drive/MyDrive/ML/cv/covid/COVID-444.png
/content/drive/MyDrive/ML/cv/covid/COVID-465.png
/content/drive/MyDrive/ML/cv/covid/COVID-470.png
/content/drive/MyDrive/ML/cv/covid/COVID-460.png
/content/drive/MyDrive/ML/cv/covid/COVID-468.png
/content/drive/MyDrive/ML/cv/covid/COVID-443.png
/content/drive/MyDrive/ML/cv/covid/COVID-459.png
/content/drive/MyDrive/ML/cv/covid/COVID-455.png
/content/drive/MyDrive/ML/cv/covid/COVID-467.png
/content/drive/MyDrive/ML/cv/covid/COVID-464.png
/content/drive/MyDrive/ML/cv/covid/COVID-45.png
/content/drive/MyDrive/ML/cv/covid/COVID-461.png
/content/drive/MyDrive/ML/cv/covid/COVID-447.png
/content/drive/MyDrive/ML/cv/covid/COVID-452.png
/content/drive/MyDrive/ML/cv/covid/COVID-454.png
/content/drive/MyDrive/ML/cv/covid/COVID-466.png
/content/drive/MyDrive/ML/cv/covid/COVID-453.png
/content/drive/MyDrive/

100%|██████████| 500/500 [00:01<00:00, 297.93it/s]

/content/drive/MyDrive/ML/cv/covid/COVID-496.png
/content/drive/MyDrive/ML/cv/covid/COVID-491.png
/content/drive/MyDrive/ML/cv/covid/COVID-489.png
/content/drive/MyDrive/ML/cv/covid/COVID-494.png
/content/drive/MyDrive/ML/cv/covid/COVID-492.png
/content/drive/MyDrive/ML/cv/covid/COVID-497.png
/content/drive/MyDrive/ML/cv/covid/COVID-486.png
/content/drive/MyDrive/ML/cv/covid/COVID-49.png
/content/drive/MyDrive/ML/cv/covid/COVID-52.png
/content/drive/MyDrive/ML/cv/covid/COVID-53.png
/content/drive/MyDrive/ML/cv/covid/COVID-54.png
/content/drive/MyDrive/ML/cv/covid/COVID-55.png
/content/drive/MyDrive/ML/cv/covid/COVID-56.png
/content/drive/MyDrive/ML/cv/covid/COVID-58.png
/content/drive/MyDrive/ML/cv/covid/COVID-57.png
/content/drive/MyDrive/ML/cv/covid/COVID-59.png
/content/drive/MyDrive/ML/cv/covid/COVID-6.png
/content/drive/MyDrive/ML/cv/covid/COVID-61.png
/content/drive/MyDrive/ML/cv/covid/COVID-60.png
/content/drive/MyDrive/ML/cv/covid/COVID-63.png
/content/drive/MyDrive/ML/cv/covid

In [137]:
temp = np.array(data_normal)
temp[5][0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


array([[[0.13333333, 0.13333333, 0.13333333],
        [0.20784314, 0.20784314, 0.20784314],
        [0.25098039, 0.25098039, 0.25098039],
        ...,
        [0.07843137, 0.07843137, 0.07843137],
        [0.1372549 , 0.1372549 , 0.1372549 ],
        [0.18039216, 0.18039216, 0.18039216]],

       [[0.08627451, 0.08627451, 0.08627451],
        [0.18039216, 0.18039216, 0.18039216],
        [0.22352941, 0.22352941, 0.22352941],
        ...,
        [0.20784314, 0.20784314, 0.20784314],
        [0.24313725, 0.24313725, 0.24313725],
        [0.28235294, 0.28235294, 0.28235294]],

       [[0.04313725, 0.04313725, 0.04313725],
        [0.14509804, 0.14509804, 0.14509804],
        [0.19607843, 0.19607843, 0.19607843],
        ...,
        [0.30588235, 0.30588235, 0.30588235],
        [0.3254902 , 0.3254902 , 0.3254902 ],
        [0.32941176, 0.32941176, 0.32941176]],

       ...,

       [[0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        ],
        [0.        , 0

In [ ]:
from numpy import save
save('/content/drive/MyDrive/ML/cv/covid.npy', data_covid)
save('/content/drive/MyDrive/ML/cv/normal.npy', data_normal)

In [30]:
covid_drive = load(file = "/content/drive/MyDrive/ML/cv/covid.npy", allow_pickle= True )
normal_drive = load('/content/drive/MyDrive/ML/cv/normal.npy', allow_pickle= True )

In [68]:
train = np.concatenate((covid_drive[:400], normal_drive[:400]))
test = np.concatenate((covid_drive[400:], normal_drive[400:]))
shuffle(train)

In [69]:
train.shape, test.shape

((800, 2), (200, 2))

In [81]:
X_train = [1]*train.shape[0]
Y_train = [1]*train.shape[0]
for i in range(train.shape[0]) :
  X_train[i] = train[0][0]
  Y_train[i] = train[0][1]

print(X_train[:2][:2])
print(Y_train[:5][:])

[array([[  1,   0,   0, ...,   0, 186, 197],
       [ 79,  75,  74, ...,   0, 191, 201],
       [ 59,  81,  25, ...,   0, 192, 200],
       ...,
       [129,  70,  33, ...,   0,   0,   1],
       [122,  87,  27, ...,   0,   0,   1],
       [125,  93,  33, ...,   0,   0,   1]], dtype=uint8), array([[  1,   0,   0, ...,   0, 186, 197],
       [ 79,  75,  74, ...,   0, 191, 201],
       [ 59,  81,  25, ...,   0, 192, 200],
       ...,
       [129,  70,  33, ...,   0,   0,   1],
       [122,  87,  27, ...,   0,   0,   1],
       [125,  93,  33, ...,   0,   0,   1]], dtype=uint8)]
[array(0), array(0), array(0), array(0), array(0)]


In [82]:
X_test = [1]*test.shape[0]
Y_test = [1]*test.shape[0]
for i in range(test.shape[0]) :
  X_test[i] = test[0][0]
  Y_test[i] = test[0][1]

print(X_test[:2][:2])
print(Y_test[:5][:])

[array([[  6,   3,   2, ...,   3,   3,  11],
       [  5,   2,   3, ...,   5,   0,  15],
       [  6,   2,   1, ...,  11,  10,   8],
       ...,
       [255, 252, 253, ..., 118, 102,  88],
       [254, 254, 255, ..., 117, 100,  84],
       [253, 253, 254, ..., 119, 100,  82]], dtype=uint8), array([[  6,   3,   2, ...,   3,   3,  11],
       [  5,   2,   3, ...,   5,   0,  15],
       [  6,   2,   1, ...,  11,  10,   8],
       ...,
       [255, 252, 253, ..., 118, 102,  88],
       [254, 254, 255, ..., 117, 100,  84],
       [253, 253, 254, ..., 119, 100,  82]], dtype=uint8)]
[array(0), array(0), array(0), array(0), array(0)]


In [ ]:
model.fit(X_train, Y_train, epochs = 25, batch_size = 32)